In [1]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [4]:
import datasets
import numpy as np
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification

In [5]:
from datasets import load_from_disk
train_dataset=load_from_disk('/content/drive/MyDrive/NER_datasets/encoded_train')

In [6]:
from transformers import RobertaModel, RobertaTokenizerFast

# Load pre-trained RoBERTa model for token classification
model_name = "roberta-base"
base_model = RobertaModel.from_pretrained(model_name)
tokenizer = RobertaTokenizerFast.from_pretrained(model_name,add_prefix_space=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
train_dataset[0]

{'tokens': ['steve',
  'mcqueen',
  'provided',
  'a',
  'thrilling',
  'motorcycle',
  'chase',
  'in',
  'this',
  'greatest',
  'of',
  'all',
  'ww',
  '2',
  'prison',
  'escape',
  'movies'],
 'ner_tags': [1, 2, 0, 0, 9, 10, 10, 10, 10, 15, 16, 16, 9, 10, 10, 10, 10]}

In [8]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        label_ids = []
        # Special tokens like `` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [9]:
tokenized_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)

In [10]:
tokenized_dataset

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 7816
})

In [11]:
tokenized_dataset.save_to_disk("/content/drive/MyDrive/NER_datasets/tokenized_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/7816 [00:00<?, ? examples/s]

In [13]:
datacollator=DataCollatorForTokenClassification(tokenizer)

In [14]:
!pip install seqeval
metric=datasets.load_metric("seqeval")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=d264c18e85016e8dbb17c7dfa4420d9c4ad56f80524893837c805570efa952f3
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


<ipython-input-14-4eb4641685a9>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric=datasets.load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [15]:
label_list=['O', 'B-Actor', 'I-Actor', 'B-Character_Name', 'I-Character_Name', 'B-Director', 'I-Director', 'B-Genre', 'I-Genre', 'B-Plot', 'I-Plot', 'B-Year', 'I-Year', 'B-Soundtrack', 'I-Soundtrack', 'B-Opinion', 'I-Opinion', 'B-Award', 'I-Award', 'B-Origin', 'I-Origin', 'B-Quote', 'I-Quote', 'B-Relationship', 'I-Relationship']


In [16]:
import torch
def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)

    return {
          "precision": results["overall_precision"],
          "recall": results["overall_recall"],
          "f1": results["overall_f1"],
          "accuracy": results["overall_accuracy"],
  }


In [17]:
test_dataset=load_from_disk("/content/drive/MyDrive/NER_datasets/encoded_test")

In [18]:
test_tokenized_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1953 [00:00<?, ? examples/s]

In [19]:
torch.cuda.is_available()

True

In [20]:
base_model.config.hidden_size

768

In [21]:

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class CustomRoBERTaForTokenClassification(nn.Module):
    def __init__(self,num_labels):
        super(CustomRoBERTaForTokenClassification, self).__init__()
        self.roberta = base_model
        self.dropout = nn.Dropout(0.1)
        #classifier
        self.classifier = nn.Linear(768,num_labels)
        self.num_labels = num_labels

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        #passing through classifier
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return (loss, logits) if loss is not None else logits

num_labels = 25
model = CustomRoBERTaForTokenClassification(num_labels).to(device)


In [22]:
#Define training args
from transformers import TrainingArguments, Trainer

In [23]:
args = TrainingArguments(
"test-ner",
eval_strategy = "epoch",
learning_rate=2e-5,
per_device_train_batch_size=16,
per_device_eval_batch_size=4,
num_train_epochs=15,
weight_decay=0.04,

)

In [24]:
trainer=Trainer(model,
                args,
                train_dataset=tokenized_dataset,
                eval_dataset=test_tokenized_dataset,
                data_collator=datacollator,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics
                )

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.394199,0.690603,0.751452,0.719744,0.877326
2,0.585000,0.362067,0.721829,0.759278,0.740080,0.880134
3,0.332700,0.371053,0.728960,0.778465,0.752900,0.887485
4,0.270300,0.377303,0.720622,0.783388,0.750696,0.886563
5,0.231500,0.411354,0.735680,0.778213,0.756349,0.886212
6,0.202800,0.403798,0.734192,0.769503,0.751433,0.883074
7,0.179500,0.430982,0.738717,0.774804,0.756330,0.885576
8,0.157400,0.450062,0.744365,0.779601,0.761576,0.884983
9,0.135500,0.480701,0.743056,0.769881,0.756231,0.884808
10,0.120000,0.500464,0.739871,0.779096,0.758977,0.885137


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=7335, training_loss=0.18457856155530725, metrics={'train_runtime': 1532.2963, 'train_samples_per_second': 76.513, 'train_steps_per_second': 4.787, 'total_flos': 0.0, 'train_loss': 0.18457856155530725, 'epoch': 15.0})

In [26]:
predictions = trainer.predict(test_tokenized_dataset)

In [54]:
metrics=trainer.evaluate(test_tokenized_dataset)
print("Accuracy:",metrics['eval_accuracy'])
print("Precision:",metrics['eval_precision'])
print("Recall:",metrics['eval_recall'])
print("F1_score:",metrics['eval_f1'])

Accuracy: 0.8813629980691592
Precision: 0.7333892014844966
Recall: 0.7732895733400657
F1_score: 0.7528110599078341


In [27]:
torch.save(model.state_dict(),"/content/drive/MyDrive/NER_weights_movie/nerweightrobertacustom2")

In [49]:
id2label = {
    i: label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [50]:
id2label

{0: 'O',
 1: 'B-Actor',
 2: 'I-Actor',
 3: 'B-Character_Name',
 4: 'I-Character_Name',
 5: 'B-Director',
 6: 'I-Director',
 7: 'B-Genre',
 8: 'I-Genre',
 9: 'B-Plot',
 10: 'I-Plot',
 11: 'B-Year',
 12: 'I-Year',
 13: 'B-Soundtrack',
 14: 'I-Soundtrack',
 15: 'B-Opinion',
 16: 'I-Opinion',
 17: 'B-Award',
 18: 'I-Award',
 19: 'B-Origin',
 20: 'I-Origin',
 21: 'B-Quote',
 22: 'I-Quote',
 23: 'B-Relationship',
 24: 'I-Relationship'}

In [32]:
from transformers import pipeline

In [58]:
# Example text for inference
text1="Another movie that i say i should've watched before. Intense romantic drama. Fascinating portrait of loneliness in a post-digital world. Amazingly original screenplay, concept, musics, especially colors and performance from Joaquin Phoenix."
text2="An intriguing, interesting and original story focussing on Louise Banks (Amy Adams) a preeminent linguistics professor who must lead humanity in communicating with visiting aliens. I think what most caught me surprised that it's so different from all the other scifi alien attacks movies - it's not all action all-guns-blazing at the threat - yet, rather a thoughtful humanity driven movie. Amy Adams is amazing as the protagonist demonstrating humanity as well communicating and befriending the aliens' unknown language and the impact it has on her. She carries the movie single-handed. I think it's a very beautiful message about humanity, what's the whole story is about. And tragic, surely, that despite knowing her future and the painful future she will have, she will not change anything. That's a great message. Despite knowing the journey, and where it leads... I embrace it... and I welcome every moment of it."
new_texts = [text1,text2]

# Tokenize the new text
inputs = tokenizer(new_texts, padding=True, truncation=True, return_tensors="pt")

input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

# Set the model to evaluation mode
model.eval()

# Make predictions
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
predicted_labels=[]
# Process the logits to get predicted labels
for i in range(len(new_texts)):
  logits = outputs[i]
  predicted_label = torch.argmax(logits, dim=-1).cpu().numpy()
  predicted_labels.append(predicted_label)

# Decode the token IDs to words
decoded_texts = tokenizer.batch_decode(input_ids.cpu().numpy(), skip_special_tokens=False)

# Get the actual words for each token
tokens = [tokenizer.convert_ids_to_tokens(ids, skip_special_tokens=False) for ids in input_ids.cpu().numpy()]

# Print the words and predicted labels
for text_tokens, labels in zip(tokens, predicted_labels):
    for token, label in zip(text_tokens, labels):
        # Skip padding tokens
        if token not in tokenizer.all_special_tokens:
            print(f"Token: {token}, Label: {id2label[label]}")
    print('\nNext Sentence\n')


Token: ĠAnother, Label: O
Token: Ġmovie, Label: O
Token: Ġthat, Label: O
Token: Ġi, Label: O
Token: Ġsay, Label: O
Token: Ġi, Label: I-Opinion
Token: Ġshould, Label: I-Opinion
Token: 've, Label: I-Opinion
Token: Ġwatched, Label: I-Opinion
Token: Ġbefore, Label: I-Opinion
Token: ., Label: O
Token: ĠInt, Label: B-Opinion
Token: ense, Label: B-Opinion
Token: Ġromantic, Label: B-Genre
Token: Ġdrama, Label: I-Genre
Token: ., Label: O
Token: ĠFasc, Label: B-Opinion
Token: inating, Label: B-Opinion
Token: Ġportrait, Label: B-Plot
Token: Ġof, Label: I-Plot
Token: Ġloneliness, Label: I-Plot
Token: Ġin, Label: I-Plot
Token: Ġa, Label: I-Plot
Token: Ġpost, Label: I-Plot
Token: -, Label: I-Plot
Token: digital, Label: I-Plot
Token: Ġworld, Label: I-Plot
Token: ., Label: O
Token: ĠAmazing, Label: B-Opinion
Token: ly, Label: B-Opinion
Token: Ġoriginal, Label: I-Opinion
Token: Ġscreenplay, Label: O
Token: ,, Label: O
Token: Ġconcept, Label: O
Token: ,, Label: O
Token: Ġmus, Label: O
Token: ics, Label: